<a href="https://colab.research.google.com/github/MuhammadDanyialKhan/Google-Colaboratory-Notebooks/blob/master/Fasion_MNIST_DS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Dependencies


In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

# import tensorflow and tfds
import tensorflow as tf
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

# import helper libs
import math
import numpy as np
import matplotlib.pyplot as plt

# print tf version
print(tf.__version__)

tf.enable_eager_execution()

1.14.0


In [0]:
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

## Import the Fashion MNIST dataset

In [6]:
dataset, metadata = tfds.load('fashion_mnist', as_supervised=True, with_info=True)
Train_dataset, Test_dataset = dataset['train'], dataset['test']

W0802 18:17:33.816692 139909242742656 dataset_builder.py:439] Warning: Setting shuffle_files=True because split=TRAIN and shuffle_files=None. This behavior will be deprecated on 2019-08-06, at which point shuffle_files=False will be the default for all splits.


Dataset fashion_mnist downloaded and prepared to /root/tensorflow_datasets/fashion_mnist/1.0.0. Subsequent calls will reuse this data.
